In [1]:
import numpy as np
import pandas as pd
import recsys as rs
import json
import notipy
import random
import TopSimilarRecommender as TSR

In [2]:
train = pd.read_csv('Data/train_final.csv','\t')
tr_info = pd.read_csv('Data/tracks_final.csv','\t')
tgt_pl = pd.read_csv('Data/target_playlists.csv','\t')
tgt_tr = pd.read_csv('Data/target_tracks.csv','\t')

In [3]:
big_target_playlist = [i for i in np.unique(train['playlist_id'].values) if train[train['playlist_id'] == i].shape[0] >= 10]

In [4]:
test_pl = pd.DataFrame({"playlist_id": list(map(lambda x: random.choice(big_target_playlist), range(100)))})

In [5]:
tracks_in_test_pl = pd.merge(train, test_pl, how='inner', on='playlist_id')
tracks_removed= pd.DataFrame(columns=['playlist_id', 'track_id'], dtype='int32')
indexes_to_remove= np.array([], dtype='int32')
for i in np.unique(tracks_in_test_pl['playlist_id'].values):
    #randomly selects 5 elements from playlist i and save indexes
    tmp = (tracks_in_test_pl.where(tracks_in_test_pl['playlist_id']==i).dropna().sample(5)).index
    #insert the items selected in tracks_removed dataFrame
    tracks_removed= tracks_removed.append(tracks_in_test_pl.take(tmp)).astype('int64')
    #remove tracks from the original dataframe
    indexes_to_remove=np.append(indexes_to_remove, values=tmp)

tracks_in_train= pd.DataFrame(tracks_in_test_pl)
tracks_in_train.drop(indexes_to_remove, inplace=True)
tracks_removed

playlist_id  track_id
3994       169003   3666749
3988       169003   1908969
3991       169003    180213
3995       169003    426762
3981       169003   2420944
3976       170920   2040538
3975       170920   2782864
3969       170920    628730
3972       170920   2830848
3973       170920    727904
526        200629   3450564
479        200629   3542470
503        200629   1863934
521        200629   3755212
515        200629    820242
370        241061    287210
365        241061   3581156
374        241061   1292066
368        241061   1258768
377        241061   1896446
4007       285803   1522750
4008       285803    723768
4013       285803   3629119
4009       285803   2701007
4012       285803   3714490
3517       376810    727895
3518       376810   1470097
3531       376810    652718
3538       376810    858614
3539       376810    697954
...           ...       ...
11       11505740   1719272
55       11505740   2809574
63       11505740    858336
7        11505740    727682
14       11505740   2893246
2985     11541825   2185355
2966     11541825   2525701
2970     11541825   2438352
2962     11541825   3703412
2957     11541825   2154701
3707     11594877   3821070
3714     11594877   2748792
3719     11594877   1942121
3720     11594877   1909019
3705     11594877   2152563
1311     11662503   1261800
1315     11662503   1963087
1312     11662503   3351739
1314     11662503    647961
1319     11662503   3837178
3766     11725276    870203
3758     11725276    128387
3765     11725276   1881845
3770     11725276    418588
3761     11725276   2705249
3007     11742550   3141836
3010     11742550    294490
2999     11742550   1700871
3002     11742550   1850807
3005     11742550   3035866

[490 rows x 2 columns]

In [ ]:
tgt_test_tracks= pd.DataFrame(tracks_in_test_pl['track_id'])
print(tgt_test_tracks.shape)
tgt_test_tracks = pd.DataFrame.drop_duplicates(tgt_test_tracks)
print(tgt_test_tracks.shape)

In [3]:
fit_dict = {'tracks_info' : tr_info, 
            'attributes' : ['artist_id', 'album', 'tags'],
            'tgt_tracks' : tgt_test_tracks,
            'n_min_attr' : 2,
            'measure' : 'dot',
            'shrinkage' : 0,
            'n_el_sim' : 20}

recommend_dict = {'tgt_playlists' : test_pl,
                  'train_playlists_tracks_pairs' : tracks_in_test_pl, 
                  'normalize' : False}

In [4]:
rec = TSR.TopSimilarRecommender()
rec.fit(**fit_dict)
notipy.notify('Model fitted!')
print('Model fitted!')

Fixed dataset
Calculated Indices
Pruning attributes.
Number of attributes before pruning is: 120295
Number of attributes after pruning is: 35250
ICM built
Diagonal to 0
Computed 0 similarities over 100000 with dot measure and 0 shrinkage.
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diago

In [ ]:
recommendetions = rec.recommend(**recommend_dict)
notipy.notify('Recommending completed!')
print('Recommending completed!')

In [ ]:
map_eval = rs.evaluate(recommendetions, tracks_removed, 'MAP')
notipy.notify('Evaluation completed!')
print('Evaluation completed!')

In [ ]:
run_data = {'recommender_type' : rec.__class__.__name__,
            'fit_parameters' : {'attributes' : fit_dict['attributes'],
                                'n_min_attr' : fit_dict['n_min_attr'],
                                'measure' : fit_dict['measure'],
                                'shrinkage' : fit_dict['shrinkage'],
                                'n_el_sim' : fit_dict['n_el_sim']},
            'recommend_parameters' : {'normalize' : recommend_dict['normalize']},
            'evaluation_result' : 0}

In [ ]:
with open('runs_data.json', 'w') as fp:
    json.dump(run_data, fp)
notipy.notify('Run data saved!')
print('Run data saved!')

In [ ]:
recommendetions.to_csv('Submissions/top_similar_submission_' + str(map_eval), index=False)
notipy.notify('Results saved as csv!')
print('Results saved as csv!')